In [1]:
import pandas as pd
import numpy as np
from IPython.display import display_markdown
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None) 

<div class="alert alert-block alert-success">     <font size=3>
<b>Ограничения</b>
<li> Даты включениЯ: <br>
- Дата включения в реестр пациентов, нуждающихся в ПМП<br>
- Дата постановки на учет/включения пациента в реестр<br>
- Дата признания пациента нуждающимся в ПМП (ВК)<br>
    - Дата регистрации
<li> Если нет даты включения - не берем такие записи
<li>Только УМЕРШИЕ 
<li>берем УМЕРШИХ в периоде рассмотрения 
<li> Отрицат откидываем 

<div class="alert alert-block alert-warning">  <font size=3>
<b> Функции для расчета  <br>
</b>            </div>

<div class="alert alert-block alert-info">
функция возраста

In [3]:
# -------------AGE -------------
def calculate_age(x,y):
    age = y.year - x.year - ((y.month, y.day) < (x.month, x.day)) 
    if age < 0:
        age = 0
    return age

#### функция 605го

In [4]:
# def load_data_605(file_name, bmchu, bk, date_finish): -- изменения в подсчете ВОЗРАСТА -------------- 06.10.23 ----- возраст считаем для момента ВХОДА в ПЕРИОД (за время рассмотрения дети взрослеют и на конец периода мы их можем не поймать) --------
def load_data_605(file_name, bmchu, bk, date_start):
        
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла 605го ----------------------
    
    # file_name = file_605
    df = pd.read_csv(file_name, delimiter=';', encoding='cp1251')

    # уберем лишние колонки
    df_605 = df.copy()
    df_605 = df_605[[
                     'Дата включения в реестр пациентов, нуждающихся в ПМП',
                     'Дата постановки на учет/включения пациента в реестр',
                     'Дата признания пациента нуждающимся в ПМП (ВК)',
                     'Дата регистрации',
           'Пациент', 
           'Дата рождения', 'Пол',
           
           'Текущий статус пациента',
           'Дата прекращения оказания ПМП', 
             'Курирующее подразделение/филиал',
                   'Гражданский статус' ,
                    'Адрес', 'Основной диагноз по МКБ-10',
                     'Адрес фактический',
                     'Дата разрешения ДЗМ',
                     'Номер разрешения ДЗМ',
                     'БК',
                     'Наблюдение в РЦ'
                    ]]

    #  переименуем покороче столбцы
    df_605.rename(columns={'Дата включения в реестр пациентов, нуждающихся в ПМП': 'Дата_вкл_реестр_гор'}, inplace=True)
    df_605.rename(columns={'Дата постановки на учет/включения пациента в реестр': 'Дата_вкл_реестр_цпп'}, inplace=True)
    df_605.rename(columns={'Дата признания пациента нуждающимся в ПМП (ВК)': 'Дата_признания'}, inplace=True)
    df_605.rename(columns={'Дата регистрации': 'Дата_регистр_в_1с'}, inplace=True)
    df_605.rename(columns={'Дата рождения': 'ДР'}, inplace=True)
    df_605.rename(columns={'Дата прекращения оказания ПМП': 'Дата_прекр'}, inplace=True)
    df_605.rename(columns={'Курирующее подразделение/филиал': 'Филиал'}, inplace=True)
    df_605.rename(columns={'Основной диагноз по МКБ-10': 'Диагноз'}, inplace=True)
    df_605.rename(columns={'Гражданский статус': 'Иногород'}, inplace=True)

    df_concat = pd.concat([df_605])


    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_вкл_реестр_гор'] = pd.to_datetime(df_concat['Дата_вкл_реестр_гор'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_вкл_реестр_цпп'] = pd.to_datetime(df_concat['Дата_вкл_реестр_цпп'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_регистр_в_1с']   = pd.to_datetime(df_concat['Дата_регистр_в_1с'], dayfirst=True, errors = 'coerce').dt.normalize()   
    df_concat['Дата_признания']      = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр']          = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()
    
    
    # Уберем записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'
    list_status_ubiraem = ['новый пациент', 'вне реестра', 'врач КЦ', 'Новый пациент', 'Умер без п/о', 'Вне реестра', 'Врач КЦ', 'умер без п/о'] # -----------------записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'----------------
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isin(list_status_ubiraem))]   
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isna())]  # - удаляем строки. в которых текущий статус не заполнен =  nan


    # ---------------------------------------------------------Уберем 'БМЧУ "Детский хоспис"' из наших данных ----------------------------
    if bmchu != 'учитываем':
        df_concat = df_concat.loc[df_concat['Филиал'] != 'БМЧУ "Детский хоспис"']  # - удаляем строки. в которых текущий статус не заполнен =  nan
        
        
   #     # ---------------------------------------------------------Уберем 'БМЧУ "Детский хоспис"' из наших данных ----------------------------
    if bk != 'с учетом БК':
        df_concat = df_concat.loc[df_concat['БК'] != 'Да']  # - удаляем строки. в которых текущий статус не заполнен =  nan     

    # УДАЛИМ полные дубликаиы строк (оставляем только ПЕРВЫЕ строки из повторов (из 605го) )
    df_concat = df_concat.drop_duplicates(keep='first')
    df_concat.index.duplicated()
    df_concat.reset_index(drop= True , inplace= True )


    # Заполним поле Дата_первых_сведений данными:  -------------------------  ПО ОГРАНИЧЕНИЯМ!!!!!!!! см.шапку
    df_concat['Дата_первых_сведений'] = np.nan
    df_concat.loc[ (~df_concat['Дата_вкл_реестр_цпп'].isna()) , 'Дата_первых_сведений'] = df_concat['Дата_вкл_реестр_цпп']
    df_concat.loc[ (df_concat['Дата_вкл_реестр_цпп'].isna()) & (~df_concat['Дата_вкл_реестр_гор'].isna()) , 'Дата_первых_сведений'] = df_concat['Дата_вкл_реестр_гор']
    df_concat.loc[ (df_concat['Дата_вкл_реестр_цпп'].isna()) & (df_concat['Дата_вкл_реестр_гор'].isna()) & (~df_concat['Дата_признания'].isna())  , 'Дата_первых_сведений'] = df_concat['Дата_признания']
    
    # df_concat.loc[  (df_concat['Дата_вкл_реестр_цпп'].isna()) 
    #               & (df_concat['Дата_вкл_реестр_гор'].isna()) 
    #               & (df_concat['Дата_признания'].isna()) 
    #               & ((~df_concat['Дата_регистр_в_1с'].isna()) & (df_concat['Дата_регистр_в_1с'] > '2023-01-01'))
    #                  , 'Дата_первых_сведений'] = df_concat['Дата_регистр_в_1с']
    
    df_concat.loc[  (df_concat['Дата_вкл_реестр_цпп'].isnull()) 
              & (df_concat['Дата_вкл_реестр_гор'].isnull()) 
              & (df_concat['Дата_признания'].isnull()) 
              & (~df_concat['Дата_регистр_в_1с'].isnull())
              & (df_concat['Дата_регистр_в_1с'] > '2023-01-01')
                  , 'Дата_первых_сведений'] = df_concat['Дата_регистр_в_1с']
    
    
    # df_concat.loc[ (~df_concat['Дата_вкл_реестр_цпп'].isnull()) & (~df_concat['Дата_вкл_реестр_цпп'].isnull()) & (~df_concat['Дата_признания'].isnull()) 
    #                & (df_concat['Дата_вкл_реестр_цпп'] < df_concat['Дата_признания'])  , 'Дата_первых_сведений'] = df_concat['Дата_признания']

    # --------------------- выкинуть пустые если даты первых нет !!!! --------------
    df_concat = df_concat[~df_concat['Дата_первых_сведений'].isna()]

    df_concat['Дата_первых_сведений']          = pd.to_datetime(df_concat['Дата_первых_сведений'], dayfirst=True, errors = 'coerce').dt.normalize()  

    # новый признак - Дата_выбытия
    df_concat['Дата_выбытия'] = np.nan
    # 1 -- условие ---------------------------
    df_concat.loc[ (  
               ((df_concat['Дата_прекр'].isnull()) & (df_concat['Дата_первых_сведений'] < '2020-12-31'))  
            &  (  (df_concat['Текущий статус пациента'] == 'Умер') | (df_concat['Текущий статус пациента'] == 'Прекращено оказание ПМП') )
               )
               , 'Дата_выбытия'] = '01.01.1900'

    # 2 -- условие----------------------------
    df_concat.loc[(~df_concat['Дата_прекр'].isnull()), 'Дата_выбытия'] = df_concat['Дата_прекр']

    # # 3 -- условие ---------------------------
    # df_concat.loc[
    #                  (~df_concat['Дата_первых_сведений'].isnull() & df_concat['Дата_выбытия'].isnull() )  
    #               &  ((df_concat['Текущий статус пациента'] == 'Умер') | (df_concat['Текущий статус пациента'] == 'Прекращено оказание ПМП')), 'Дата_выбытия'] = df_concat['Дата_первых_сведений'] + pd.Timedelta(days = 2)

    
    # df_concat = df_concat[~df_concat['Дата_выбытия'].isna()]
                     

    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    # df_concat['Дата_вкл_реестр'] = pd.to_datetime(df_concat['Дата_вкл_реестр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_признания'] = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр'] = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_выбытия'] = pd.to_datetime(df_concat['Дата_выбытия'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_первых_сведений'] = pd.to_datetime(df_concat['Дата_первых_сведений'], dayfirst=True, errors = 'coerce').dt.normalize()
    
#     df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
#     df_concat['Дата_вкл_реестр_гор'] = pd.to_datetime(df_concat['Дата_вкл_реестр_гор'], dayfirst=True, errors = 'coerce').dt.normalize()
#     df_concat['Дата_вкл_реестр_цпп'] = pd.to_datetime(df_concat['Дата_вкл_реестр_цпп'], dayfirst=True, errors = 'coerce').dt.normalize()
#     df_concat['Дата_регистр_в_1с']   = pd.to_datetime(df_concat['Дата_регистр_в_1с'], dayfirst=True, errors = 'coerce').dt.normalize()   
#     df_concat['Дата_признания']      = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
#     df_concat['Дата_прекр']          = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()
    
    
    

   #  # ------------подсчитаем ВОЗРАСТ --------------
   # # now = pd.Timestamp('now')
   #  now = pd.Timestamp(date_finish)
   #  df_concat.loc[(df_concat['Текущий статус пациента'] == 'Умер') & (~df_concat['Дата_выбытия'].isnull()), 'Возраст'] = (df_concat['Дата_выбытия'] - df_concat['ДР']).dt.days//365
   #  df_concat.loc[(df_concat['Текущий статус пациента'] != 'Умер'), 'Возраст'] = (now - df_concat['ДР']).astype('<m8[Y]')

   #    ------------------------- ---подсчитаем ВОЗРАСТ -------------- 06.10.23 ----- возраст считаем для момента ВХОДА в ПЕРИОД (за время рассмотрения дети взрослеют и на конец периода мы их можем не поймать) ---------------
    
   # now = pd.Timestamp('now')
    now = pd.Timestamp(date_start)
    # df_concat.loc[(df_concat['Текущий статус пациента'] == 'Умер') & (~df_concat['Дата_выбытия'].isnull()), 'Возраст'] = (df_concat['Дата_выбытия'] - df_concat['ДР']).dt.days//365
    # df_concat.loc[(df_concat['Текущий статус пациента'] != 'Умер'), 'Возраст'] = (pd.Timestamp(date_start) - df_concat['ДР']).astype('<m8[Y]')
    
    df_concat['Возраст'] = np.nan
    df_concat['Возраст'] = df_concat.apply( lambda x:  calculate_age(x['ДР'], now),axis=1)
   


    #  ------ посчитаем категорию трудоспособный --------
    #  если пол ЖЕНСКИЙ, и возраст Менее 57 -----> Категория - трудоспособный
    #  если пол Мужской, и возраст Менее 62 -----> Категория - трудоспособный

    df_concat['Категория'] = np.nan
    df_concat.loc[(df_concat['Возраст'] >= 18) & (df_concat['Пол'] == 'Мужской') & (df_concat['Возраст'] >= 62), 'Категория'] = 'Старше трудосп'
    df_concat.loc[(df_concat['Возраст'] >= 18) & (df_concat['Пол'] == 'Женский') & (df_concat['Возраст'] >= 57), 'Категория'] = 'Старше трудосп'


    #  ------ определим детей ---------------
    df_concat['Дети'] = np.nan
    # df_concat.loc[(df_concat['Возраст'] < 18) & (df_concat['Возраст'] >= 0), 'Дети'] = 'Дети'
    df_concat['Дети'] = df_concat['Возраст'].apply(lambda x: 'Дети' if x <= 17 else np.nan)


    #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------

    df_concat['Ключ'] = df_concat['Пациент'].map(str) + (df_concat['ДР'].dt.date).map(str)
    df_concat['Ключ'] = df_concat['Ключ'].str.lower()            # ---- все к нижнему регистру
    df_concat['Ключ'] = df_concat['Ключ'].str.replace(' ', '')   # ---- пробелы долой 


    df_clr = df_concat.loc[ (~df_concat['Дата_первых_сведений'].isnull())]

    # ------ удаление дубликатов ----------
    df_clr = df_clr.drop_duplicates(['Ключ'], keep='first') 
    
    # ------ УДАЛИМ скобку  ----------
    df_clr['Диагноз'] = df_clr['Диагноз'].str.replace('(','')
    
     # ------ Добавление краткого диагноза ----------
    df_clr['Кр_диаг'] = df_clr['Диагноз'].str[:3]
    
  
    
    return df_clr  

<div class="alert alert-block alert-success"><font size=3>
<b> Расчет сведений по ДИАГНОЗАМ из 605го <br>

<div class="alert alert-block alert-info"><font size=3>
<b>Распишем ДИАГНОЗЫ (внесем новые колонки-принаки в датафрейм)</b>               

In [5]:
# ----------- загрузка диагнозов

def load_list_diagnoz(file_diagnoz):
    
    df = pd.read_csv(file_diagnoz, delimiter=';', encoding='cp1251')

    #     ------------- сформировали краткий список ---------------
    df['Кр_диаг'] = df['Основной диагноз по МКБ-10'].str[:3]
    
    #     ------------- Удалим повторные диагноы в СПИСКЕ ДИАГНОЗОВ  ---------------
    df = df.drop_duplicates('Кр_диаг')
    
    return df


<hr>

<font color= green size=10>__ВЫВОД РЕЗУЛЬТАТОВ ПО ДИАГНОЗАМ__
<hr>

In [6]:
# df_reestr = load_data_605(file_name, bmchu, bk,start)

In [7]:
# ---------------------------- Применение функций ---------------
start = '2021-01-01'
finish = '2024-12-31'


# file_name = 'выгрузка_605_из_1с_0610.csv'
file_name = 'выгрузка_из_1с_20240219_0837.csv'
# file_diag = 'список диагнозов.12,01.2024.csv'
file_diag = 'список диагнозов_12.01.2024_vs_old.csv'
bmchu = 'не учитываем'
# bmchu = 'учитываем'
bk = 'с учетом БК'
# bk = 'без учета БК'
# ---------------------------- Готовим датафрейм РЕЕСТРА ---------------
df_reestr = load_data_605(file_name, bmchu, bk,start)
# ---------------------------- Готовим датафрейм c РАСПИСАНЫМИ ДИАГНОЗАМИ ---------------
df_diag = load_list_diagnoz(file_diag)
# ----- соединим диагнозы в реестре со СПИСКОМ диагнозов
df_reestr = pd.merge(df_reestr, df_diag, how='left', on='Кр_диаг')
# ----- Если диагноз не указан - аменим на ПРОЧИЕ ------------
df_reestr.loc[df_reestr['Тип_диагноза'].isna(), 'Тип_диагноза' ] = 'прочие'
all_clients_period = df_reestr[       (df_reestr['Дата_первых_сведений'] < finish)     
                           &   ( (df_reestr['Дата_выбытия'].isnull())  | (df_reestr['Дата_выбытия'] >= start)     ) ]

all_clients_period_vzros      = all_clients_period[all_clients_period['Дети'].isna()]
all_clients_period_vzros_trud = all_clients_period[all_clients_period['Категория']== 'Старше трудосп']
all_clients_period_deti       = all_clients_period[~all_clients_period['Дети'].isna()]


s_list = ['терапия', 'онкология',   'психиатрия', 'вич-инфекция', 'неврология', 'врожденные аномалии / пороки развития и хромосомные нарушения',   'прочие',   ]

# ------------------- все пациенты -------------
all_clients_period_ = []
for i_diag in s_list:
    all_clients_period_.append(all_clients_period[all_clients_period['Тип_диагноза'] == i_diag])

# ------------------- взрослые -------------
all_clients_period_взрос = []
for i_diag in s_list:
    all_clients_period_взрос.append(all_clients_period_vzros[all_clients_period_vzros['Тип_диагноза'] == i_diag])
    
# ------------------- старше труд -------------
all_clients_period_труд = []
for i_diag in s_list:
    all_clients_period_труд.append(all_clients_period_vzros_trud[all_clients_period_vzros_trud['Тип_диагноза'] == i_diag])

# ------------------- дети  -------------
all_clients_period_дети = []
for i_diag in s_list:
    all_clients_period_дети.append(all_clients_period_deti[all_clients_period_deti['Тип_диагноза'] == i_diag])
  

# -------------------новые онко -------------

new_onko_vzros = all_clients_period[
                        (all_clients_period['Тип_диагноза'] == 'онкология')
                       & (all_clients_period['Дата_первых_сведений'] >= start )
                        &     ((all_clients_period['Дети']).isna())  
                                          ].shape[0]

new_onko_deti = all_clients_period.loc[
                        (all_clients_period['Тип_диагноза'] == 'онкология')
                       & (all_clients_period['Дата_первых_сведений'] >= start )
    &     (~(all_clients_period['Дети']).isna())  
                                          ].shape[0]

a = f'Статистика по ДИАГНОЗАМ в период с {start} по {finish}:'

display_markdown(f'''
# {a}
| Параметр |Всего| Взрослые | В т.ч. трудоспособ | Дети |
|---:|:-|:-|:-|:-|
| __Всего пациентов__{bk}__                                           | {all_clients_period.shape[0]}      | {all_clients_period_vzros.shape[0]} |{all_clients_period_vzros_trud.shape[0]}  |{all_clients_period_deti.shape[0]}|
| пациенты с диагнозом __"Терапия / Педиатрия"__                      | {all_clients_period_[0].shape[0]} | {all_clients_period_взрос[0].shape[0]} |{all_clients_period_труд[0].shape[0]} |{all_clients_period_дети[0].shape[0]}|
| пациенты с диагнозом __"Онко"__                                     | {all_clients_period_[1].shape[0]} | {all_clients_period_взрос[1].shape[0]} |{all_clients_period_труд[1].shape[0]} |{all_clients_period_дети[1].shape[0]}|
| <span style="color: green;"> новые пациенты с диагнозом __"Онко"__  |   |<span style="color: green;">  __{new_onko_vzros}__ |- |<span style="color: green;"> __{new_onko_deti}__|
| пациенты с диагнозом __"Психиатрия"__                               | {all_clients_period_[2].shape[0]} | {all_clients_period_взрос[2].shape[0]} |{all_clients_period_труд[2].shape[0]} |{all_clients_period_дети[2].shape[0]}|
| пациенты с диагнозом __"ВИЧ"__                                      | {all_clients_period_[3].shape[0]} | {all_clients_period_взрос[3].shape[0]} |{all_clients_period_труд[3].shape[0]} |{all_clients_period_дети[3].shape[0]}|
| пациенты с диагнозом __"Неврология"__                               | {all_clients_period_[4].shape[0]} | {all_clients_period_взрос[4].shape[0]} |{all_clients_period_труд[4].shape[0]} |{all_clients_period_дети[4].shape[0]}|
| пациенты с диагнозом __"Врожденные"__                               | {all_clients_period_[5].shape[0]} | {all_clients_period_взрос[5].shape[0]} |{all_clients_period_труд[5].shape[0]} |{all_clients_period_дети[5].shape[0]}|
| пациенты с диагнозом __"Прочие"__                                   | {all_clients_period_[6].shape[0]} | {all_clients_period_взрос[6].shape[0]} |{all_clients_period_труд[6].shape[0]} |{all_clients_period_дети[6].shape[0]}|
| __Статус по БМЧУ__  | __{bmchu}__ | --|-- |-- ||

''', raw=True)


# Статистика по ДИАГНОЗАМ в период с 2021-01-01 по 2024-12-31:
| Параметр |Всего| Взрослые | В т.ч. трудоспособ | Дети |
|---:|:-|:-|:-|:-|
| __Всего пациентов__с учетом БК__                                           | 61374      | 60485 |50076  |889|
| пациенты с диагнозом __"Терапия / Педиатрия"__                      | 17464 | 17387 |15735 |77|
| пациенты с диагнозом __"Онко"__                                     | 37532 | 37440 |30140 |92|
| <span style="color: green;"> новые пациенты с диагнозом __"Онко"__  |   |<span style="color: green;">  __33666__ |- |<span style="color: green;"> __85__|
| пациенты с диагнозом __"Психиатрия"__                               | 88 | 83 |73 |5|
| пациенты с диагнозом __"ВИЧ"__                                      | 15 | 15 |0 |0|
| пациенты с диагнозом __"Неврология"__                               | 4448 | 3893 |2903 |555|
| пациенты с диагнозом __"Врожденные"__                               | 132 | 11 |0 |121|
| пациенты с диагнозом __"Прочие"__                                   | 1695 | 1656 |1225 |39|
| __Статус по БМЧУ__  | __не учитываем__ | --|-- |-- ||



<div class="alert alert-block alert-warning"> <font size=3>
<b>СЧИТАЕМ ДЛЯ ВСЕХ ПАЦИЕНТОВ - КОТОРЫЕ БЫЛИ ЗАСВЕЧЕНЫ В ЭТОМ ГОДУ НАБЛЮДЕНИЯ

In [8]:
# df_all_21 = all_clients_period
df_all_21_24 = all_clients_period.copy()
df_all_21_24.shape

(61374, 29)

In [9]:
df_all_21_24['Текущий статус пациента'].unique()

array(['Умер', 'прекращено оказание ПМП', 'патронаж',
       'временное прекращение патронажей', 'стационар ЦПП',
       'стационар ОПМП'], dtype=object)

In [10]:
# *******
# df_all_21[(df_all_21['Текущий статус пациента'] == 'Умер')  & (df_all_21['Дата_выбытия'].dt.year == 2021)].shape

<div class="alert alert-block alert-warning"> <font size=3>
Тип_диагноза онкология/не онко

In [11]:
df_all_21_24.loc[df_all_21_24['Тип_диагноза'] != 'онкология', 'Тип_диагноза'] = 'не_онко'

<div class="alert alert-block alert-warning"> <font size=3>
Считаем период наблюдения

In [12]:
df_all_21_24['прод_наблюд'] = np.nan

In [13]:
today_date = pd.to_datetime(pd.to_datetime('today').normalize(), dayfirst=True, errors = 'coerce').normalize()
# today_date

<div class="alert alert-block alert-warning"> <font size=3>
Берем только ЖИВЫЕ статусы и УМЕР в РАССМОТРЕНИЕ 

In [14]:
df_all_21_24 = df_all_21_24[df_all_21_24['Текущий статус пациента'].isin(['Умер', 'патронаж',     'стационар ЦПП',   'стационар ОПМП'])]

In [15]:
# ------------------------- ВЫКИНЕМ, если УМЕР, но даты смерти нет 
df_all_21_24 = df_all_21_24.loc[~((df_all_21_24['Текущий статус пациента'] =='Умер')
              & df_all_21_24['Дата_выбытия'].isna())]

In [16]:
# ----------------------------если дата выбытия пустая - ставим сегодня ---------------
df_all_21_24.loc[df_all_21_24['Дата_выбытия'].isna(), 'Дата_выбытия'] = today_date

In [17]:
df_all_21_24['прод_наблюд']  = df_all_21_24['Дата_выбытия']  - df_all_21_24['Дата_первых_сведений']

<div class="alert alert-block alert-warning"> <font size=3>
удалим КРИВЫЕ даты ожидания

In [18]:
# df_all_21_24.loc[df_all_21_24['прод_наблюд'] < '0 days']

In [19]:
# -------------------------- уберем отрицательные -------------------
df_all_21_24 = df_all_21_24.loc[df_all_21_24['прод_наблюд'] >= '0 days']

In [20]:
# df_all_21_24 =  df_all_21_24.loc[df_all_21_24['Текущий статус пациента'] == 'Умер']

<div class="alert alert-block alert-warning"> <font size=3>
Дети - взрос

In [21]:
df_all_21_24.loc[df_all_21_24['Дети'].isna(), 'Дети'] = 'взрослые'

<div class="alert alert-block alert-warning"> <font size=3>
Бины по дате ожидания

In [22]:
df_all_21_24['период_ожид'] = np.nan

In [23]:
# ------------------------------------Биннинг данных-----------
df_all_21_24['период_ожид'] = pd.cut(
    df_all_21_24['прод_наблюд'],
    
    bins=pd.to_timedelta(['0 days', '30 days', '90 days', '180 days', '365 days', '100000 days']),
    labels=['0-1 мес', '1-3 мес', '3-6 мес','6-12 мес','более 12']
 )

In [24]:
# df_all_21_24.head()

<div class="alert alert-block alert-warning"> <font size=3>
год_ухода

In [25]:
# ---------------------------год_ухода-----------------------
df_all_21_24['год_ухода'] = np.nan
df_all_21_24['год_ухода'] = df_all_21_24['Дата_выбытия'].dt.year

In [26]:
# df_all_21_24[df_all_21_24['Текущий статус пациента'] == 'патронаж']

<div class="alert alert-block alert-success"> <font size=3>
количество пациентов по Длительности НАБЛЮДЕНИЯ

In [27]:
df_all_21_24['год_прихода'] = df_all_21_24['Дата_первых_сведений'].dt.year

<div class="alert alert-block alert-success"> <font size=3>
<b>РАСЧЕТ ПЕРИОДА НАБЛЮДЕНИЯ!!

In [28]:
def god_rassm(god):
    df_res = df_all_21_24[(df_all_21_24['год_прихода'] <= int(god)) & (df_all_21_24['год_ухода'] >= int(god))]
    return df_res

In [29]:
df_21 = god_rassm('2021')
df_22 = god_rassm('2022')
df_23 = god_rassm('2023')

<div class="alert alert-block alert-warning"> <font size=3>
количество пациентов по Длительности НАБЛЮДЕНИЯ

In [30]:
# --------------- всего----------------
df_21[(df_21['Тип_диагноза'] == 'онкология')].groupby(['период_ожид' ]).agg({'Пациент':'count'})

,Пациент
период_ожид,
0-1 мес,4527
1-3 мес,2453
3-6 мес,1273
6-12 мес,1038
более 12,3329


In [31]:
# --------------- всего----------------
df_22[(df_22['Тип_диагноза'] == 'онкология')].groupby(['период_ожид' ]).agg({'Пациент':'count'})

,Пациент
период_ожид,
0-1 мес,4718
1-3 мес,2647
3-6 мес,1378
6-12 мес,1072
более 12,3846


In [32]:
# --------------- всего----------------
df_23[(df_23['Тип_диагноза'] == 'онкология')].groupby(['период_ожид' ]).agg({'Пациент':'count'})

,Пациент
период_ожид,
0-1 мес,5053
1-3 мес,3615
3-6 мес,2274
6-12 мес,2101
более 12,3239


<div class="alert alert-block alert-warning"> <font size=3>
количество пациентов по Длительности НАБЛЮДЕНИЯ median

df_21_дети_онко=Timedelta('211 days 00:00:00')
1257 days 12:00:00
62 days 00:00:00
446 days 00:00:00


In [33]:
# ----------------------- дети онко-----------------------
df_21_дети_онко = df_21[          (df_21['Тип_диагноза'] == 'онкология')                                  &(df_21['Дети'] == 'Дети')]['прод_наблюд'].median()
df_21_дети_онко

Timedelta('211 days 00:00:00')

In [34]:
# ----------------------- дети неонко-----------------------
df_21_дети_неонко = df_21[          (df_21['Тип_диагноза'] == 'не_онко')                                   &(df_21['Дети'] == 'Дети')]['прод_наблюд'].median()
df_21_дети_неонко

Timedelta('1257 days 12:00:00')

In [35]:
# ----------------------- взрос онко-----------------------
df_21_взрослые_онко = df_21[          (df_21['Тип_диагноза'] == 'онкология')                                   &(df_21['Дети'] == 'взрослые')]['прод_наблюд'].median()
df_21_взрослые_онко

Timedelta('62 days 00:00:00')

In [36]:
# ----------------------- взрос неонко-----------------------
df_21_взрослые_неонко = df_21[          (df_21['Тип_диагноза'] == 'не_онко')                                   &(df_21['Дети'] == 'взрослые')]['прод_наблюд'].median()
df_21_взрослые_неонко

Timedelta('446 days 00:00:00')

In [37]:
# ------------------------------------------------------------22------------------

In [38]:
# ----------------------- дети онко-----------------------
df_22_дети_онко = df_22[          (df_22['Тип_диагноза'] == 'онкология')                                   &(df_22['Дети'] == 'Дети')]['прод_наблюд'].median()
df_22_дети_онко

Timedelta('217 days 00:00:00')

In [39]:
# ----------------------- дети неонко-----------------------
df_22_дети_неонко = df_22[          (df_22['Тип_диагноза'] == 'не_онко')                                   &(df_22['Дети'] == 'Дети')]['прод_наблюд'].median()
df_22_дети_неонко

Timedelta('892 days 00:00:00')

In [40]:
# ----------------------- взрос онко-----------------------
df_22_взрослые_онко = df_22[          (df_22['Тип_диагноза'] == 'онкология')                                   &(df_22['Дети'] == 'взрослые')]['прод_наблюд'].median()
df_22_взрослые_онко

Timedelta('70 days 00:00:00')

In [41]:
# ----------------------- взрос неонко-----------------------
df_22_взрослые_неонко = df_22[          (df_22['Тип_диагноза'] == 'не_онко')
                                   &(df_22['Дети'] == 'взрослые')]['прод_наблюд'].median()
df_22_взрослые_неонко

Timedelta('417 days 00:00:00')

In [37]:
# ------------------------------------------------------------23------------------

In [43]:
# ----------------------- дети онко-----------------------
df_23_дети_онко = df_23[          (df_23['Тип_диагноза'] == 'онкология')
                                   &(df_23['Дети'] == 'Дети')]['прод_наблюд'].median()
df_23_дети_онко

Timedelta('166 days 00:00:00')

In [44]:
# ----------------------- дети неонко-----------------------
df_23_дети_неонко = df_23[          (df_23['Тип_диагноза'] == 'не_онко')
                                   &(df_23['Дети'] == 'Дети')]['прод_наблюд'].median()
df_23_дети_неонко

Timedelta('728 days 00:00:00')

In [46]:
# ----------------------- взрос онко-----------------------
df_23_взрослые_онко = df_23[          (df_23['Тип_диагноза'] == 'онкология')
                                   &(df_23['Дети'] == 'взрослые')]['прод_наблюд'].median()
df_23_взрослые_онко

Timedelta('76 days 00:00:00')

In [48]:
# ----------------------- взрос неонко-----------------------
df_23_взрослые_неонко = df_23[          (df_23['Тип_диагноза'] == 'не_онко')
                                   &(df_23['Дети'] == 'взрослые')]['прод_наблюд'].median()
df_23_взрослые_неонко

Timedelta('236 days 00:00:00')

In [63]:
print(f'---------2021---')
print(f'{str(df_21_дети_онко)=}')
print(f'{str(df_21_дети_неонко)=}')
print(f'{str(df_21_взрослые_онко)=}')
print(f'{str(df_21_взрослые_неонко)=}')
print(f'--------2022----')
print(f'{str(df_22_дети_онко)=}')
print(f'{str(df_22_дети_неонко)=}')
print(f'{str(df_22_взрослые_онко)=}')
print(f'{str(df_22_взрослые_неонко)=}')
print(f'----2023----')
print(f'{str(df_23_дети_онко)=}')
print(f'{str(df_23_дети_неонко)=}')
print(f'{str(df_23_взрослые_онко)=}')
print(f'{str(df_23_взрослые_неонко)=}')

---------2021---
str(df_21_дети_онко)='211 days 00:00:00'
str(df_21_дети_неонко)='1257 days 12:00:00'
str(df_21_взрослые_онко)='62 days 00:00:00'
str(df_21_взрослые_неонко)='446 days 00:00:00'
--------2022----
str(df_22_дети_онко)='217 days 00:00:00'
str(df_22_дети_неонко)='892 days 00:00:00'
str(df_22_взрослые_онко)='70 days 00:00:00'
str(df_22_взрослые_неонко)='417 days 00:00:00'
----2023----
str(df_23_дети_онко)='166 days 00:00:00'
str(df_23_дети_неонко)='728 days 00:00:00'
str(df_23_взрослые_онко)='76 days 00:00:00'
str(df_23_взрослые_неонко)='236 days 00:00:00'


In [49]:
df_all_21_24.to_excel('df_all_21_24.')

<div class="alert alert-block alert-warning"> <font size=3>
все НАБЛЮДЕНИЯ median

In [42]:
# -------------------------------- все ---------------
# ----------------------- дети онко-----------------------
df_21_все_онко = df_21[          (df_21['Тип_диагноза'] == 'онкология')]['прод_наблюд'].median()
df_22_все_онко = df_22[          (df_22['Тип_диагноза'] == 'онкология')]['прод_наблюд'].median()
df_23_все_онко = df_23[          (df_23['Тип_диагноза'] == 'онкология')]['прод_наблюд'].median()
print(f'{df_21_все_онко=}')
print(f'{df_22_все_онко=}')
print(f'{df_23_все_онко=}')

df_21_все_онко=Timedelta('62 days 00:00:00')
df_22_все_онко=Timedelta('70 days 00:00:00')
df_23_все_онко=Timedelta('77 days 00:00:00')
